### RBF NEURAL NETWORK TO CLASSIFY

Öncelikle RBFNN'de kullanılacak gerekli fonksiyonları tanımlamamız gerekiyor. Düzenlenmiş "kmeans" fonksiyonu, küme merkezlerini ve ayrıca kümelerin standart sapmasını döndürür.

In [1]:
import numpy as np


def get_distance(x1, x2):
    sum = 0
    for i in range(len(x1)):
        sum += (x1[i] - x2[i]) ** 2
    return np.sqrt(sum)


def kmeans(X, k, max_iters):
  
    centroids = X[np.random.choice(range(len(X)), k, replace=False)]

    converged = False
    
    current_iter = 0

    while (not converged) and (current_iter < max_iters):

        cluster_list = [[] for i in range(len(centroids))]

        for x in X:  # Go through each data point
            distances_list = []
            for c in centroids:
                distances_list.append(get_distance(c, x))
            cluster_list[int(np.argmin(distances_list))].append(x)

        cluster_list = list((filter(None, cluster_list)))

        prev_centroids = centroids.copy()

        centroids = []

        for j in range(len(cluster_list)):
            centroids.append(np.mean(cluster_list[j], axis=0))

        pattern = np.abs(np.sum(prev_centroids) - np.sum(centroids))

        print('K-MEANS: ', int(pattern))

        converged = (pattern == 0)

        current_iter += 1

    return np.array(centroids), [np.std(x) for x in cluster_list]

 Uygulamayı daha kolay hale getirmek için, RBFNN'yi bir sınıf olarak kodlayabiliriz.

In [39]:
class RBF:

    def __init__(self, X, y, tX, ty, num_of_classes,
                 k, std_from_clusters=True):
        self.X = X
        self.y = y

        self.tX = tX
        self.ty = ty

        self.number_of_classes = num_of_classes
        self.k = k
        self.std_from_clusters = std_from_clusters

    def convert_to_one_hot(self, x, num_of_classes):
        arr = np.zeros((len(x), num_of_classes))
        for i in range(len(x)):
            c = int(x[i])
            arr[i][c] = 1
        return arr

    def rbf(self, x, c, s):
        distance = get_distance(x, c)
        return 1 / np.exp(-distance / s ** 2)

    def rbf_list(self, X, centroids, std_list):
        RBF_list = []
        for x in X:
            RBF_list.append([self.rbf(x, c, s) for (c, s) in zip(centroids, std_list)])
        return np.array(RBF_list)
    
    def fit(self):

        self.centroids, self.std_list = kmeans(self.X, self.k, max_iters=1000)

        if not self.std_from_clusters:
            dMax = np.max([get_distance(c1, c2) for c1 in self.centroids for c2 in self.centroids])
            self.std_list = np.repeat(dMax / np.sqrt(2 * self.k), self.k)

        RBF_X = self.rbf_list(self.X, self.centroids, self.std_list)

        self.w = np.linalg.pinv(RBF_X.T @ RBF_X) @ RBF_X.T @ self.convert_to_one_hot(self.y, self.number_of_classes)

        RBF_list_tst = self.rbf_list(self.tX, self.centroids, self.std_list)

        self.pred_ty = RBF_list_tst @ self.w

        self.pred_ty = np.array([np.argmax(x) for x in self.pred_ty])

        diff = self.pred_ty - self.ty

        print('Accuracy: ', len(np.where(diff == 0)[0]) / len(diff))

Fit Function: İlk satırlar, ağırlık merkezlerini ve kümelerin standart sapmasını elde etmek için k-ortalamalarını gerçekleştirir. Ardından , belirtilen denklemi kullanarak algoritmanın tüm küme merkezler için aynı **Betayı** kullanmasını sağlayabiliriz . Sonraki satırlarda, X girişinin RBF'sini alıyoruz ve uygun bir ağırlık matrisi **W** elde etmek için En Küçük Kareler Optimizasyonunu uyguluyoruz . Ek olarak, modelin doğruluğunu ölçmek için son birkaç satırda test verileri kullanılır.

MNIST veri setini eğitim ve teste ayırın ve RBFNN'nin işini yapmasına izin verin.

In [4]:
data = np.load('mnist_data.npy').astype(float)

train_y = data[0:5000, 0]
train_x = data[0:5000, 1:]

test_y = data[0:1000, 0]
test_x = data[0:1000, 1:]

RBF_CLASSIFIER = RBF(train_x, train_y, test_x, test_y, num_of_classes=10,k=500, std_from_clusters=False)

RBF_CLASSIFIER.fit()

K-MEANS:  601
K-MEANS:  3
K-MEANS:  4
K-MEANS:  8
K-MEANS:  0
Accuracy:  0.975


In [15]:
data2 = np.load('data.npy').astype(float)

train_y2 = data2[0:5000, 0]
train_x2 = data2[0:5000, 2:]

test_y2 = data2[0:1000, 0]
test_x2 = data2[0:1000, 2:]

RBF_CLASSIFIER2 = RBF(train_x2, train_y2, test_x2, test_y2, num_of_classes=10,k=500, std_from_clusters=False)

RBF_CLASSIFIER2.fit()

K-MEANS:  3918
K-MEANS:  281
K-MEANS:  0
K-MEANS:  0
Accuracy:  0.9764309764309764


In [28]:
data3 = np.load('labels.npy').astype(float)

train_y3 = data3[0:5000, 0]
train_x3 = data3[0:5000, 2:]

test_y3 = data3[0:1000, 0]
test_x3 = data3[0:1000, 2:]

RBF_CLASSIFIER = RBF(train_x3, train_y3, test_x3, test_y3, num_of_classes=10,k=500, std_from_clusters=False)

RBF_CLASSIFIER.fit()

K-MEANS:  0
Accuracy:  0.6161616161616161


<ipython-input-2-5c0b36b492fb>:24: RuntimeWarning: invalid value encountered in double_scalars
  return 1 / np.exp(-distance / s ** 2)


In [6]:

# First read into csv with pandas
#data = pd.read_csv("Telco.csv")
 # Save numpy as npy
#np.save("telco.npy", data)

Sonuç olarak, RBFNN, sınıflandırma ve regresyon görevleri için güçlü modellerden biridir. RBF ağları, birçok RBF eğrisi kullanarak temeldeki örüntüleri tahmin etmeyi öğrenebilir. Optimizasyon süreci için istatistiksel denklemin uygulanması, algoritmayı MLP yapılı ağlara kıyasla daha kolay ve hızlı hale getirir . Ancak, K - küme sayısı ve Beta gibi hiperparametrelerin ince ayarı çalışma, zaman ve pratik gerektirir.

### RBF AS A KERNEL

In [36]:

from sklearn.datasets import load_iris,load_breast_cancer,load_digits,load_wine
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import train_test_split,GridSearchCV
X, y = load_iris(return_X_y=True)
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel,random_state=0).fit(X, y)
print(gpc.score(X, y))
gpc.predict_proba(X[:2,:])

0.9866666666666667


array([[0.83548752, 0.03228706, 0.13222543],
       [0.79064206, 0.06525643, 0.14410151]])

In [30]:
import numpy as np
import pandas as pd

In [31]:
from warnings import filterwarnings
filterwarnings('ignore')

In [32]:
X, y = load_breast_cancer(return_X_y=True)
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel,random_state=0).fit(X, y)
print(gpc.score(X, y))
gpc.predict_proba(X[:2,:])

0.9595782073813708


array([[0.82897859, 0.17102141],
       [0.72387831, 0.27612169]])

In [38]:
X, y = load_digits(return_X_y=True)
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel,random_state=0).fit(X, y)
print(gpc.score(X, y))
gpc.predict_proba(X[:2,:])

1.0


array([[0.10000045, 0.09999995, 0.09999995, 0.09999995, 0.09999995,
        0.09999995, 0.09999995, 0.09999995, 0.09999995, 0.09999995],
       [0.09999995, 0.10000045, 0.09999995, 0.09999995, 0.09999995,
        0.09999995, 0.09999995, 0.09999995, 0.09999995, 0.09999995]])